<a href="https://colab.research.google.com/github/CalvinXKY/mfu_calculation/blob/main/mfu_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Create a mfu calculation for LLM.
"""

In [1]:
print("test update")

test update
